#### Import libraries

In [ ]:
from datasets import load_dataset
from qdrant_client import models,QdrantClient
from sentence_transformers import SentenceTransformer

#### Load Dataset

In [ ]:
raw_datasets = load_dataset('neulab/conala',trust_remote_code=True)

In [ ]:
raw_datasets

In [ ]:
dataset = raw_datasets['train']

In [ ]:
# view dataset
dataset[0:5]

##### Coverting hugging-face dataset to pandas

In [ ]:
df_pandas = dataset.to_pandas()

In [ ]:
df_pandas.head(3)

In [ ]:
df_pandas.info()

In [ ]:
df_pandas.describe()

In [ ]:
df_pandas = df_pandas.dropna()
df_pandas.isnull().sum()

In [ ]:
# dictionary is easy formatting for creating embedding
df_dict = df_pandas.to_dict(orient='records')

#### Creating Embedding and in-memory vector databse

In [ ]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # model to create embedding

In [ ]:
# create vector database client
qdrant = QdrantClient(":memory:") # creating in-memory Qdrant instance

In [ ]:
# create collection
qdrant.recreate_collection(
    collection_name = 'conala',
    vectors_config = models.VectorParams(
        size = encoder.get_sentence_embedding_dimension(),
        distance = models.Distance.COSINE
    )
)

In [ ]:
# vectorize
qdrant.upload_points(
    collection_name='conala',
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc['intent']+" "+doc['rewritten_intent']).tolist(),
            payload=doc
        )for idx,doc in enumerate(df_dict)
    ]
)

### Take User Query

In [ ]:
user_prompt = input('Enter python coding query')

### Embed user query and perfrom retrival on vectordb

In [ ]:
hits = qdrant.search(
    collection_name='conala',
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)

In [ ]:
# define a variable to hold the search results
search_results = [hit.payload for hit in hits]

In [ ]:
print("Relevant Code Snippets:\n")
for item in search_results:
    print(f"intent: {item['rewritten_intent']} \ncode: {item['snippet']}")
    print()